### Uloha 1 - Oprava dat a spustenie algoritmu

In [1]:
import pandas as pd

In [238]:
# Načítanie dát zo súboru CSV bez hlavičky, oddeleného bodkočiarkami
data = pd.read_csv("drg-cert-2024.csv", header=None, sep=';')

In [243]:
# Prvé zobrazenie dát
data

,0,1,2,3,4,5,6,7,8
0,P019/HP663449,80,NaN,NaN,0,N178~D508~E1160~I1000~I480~N183,14n112&Z&20220209~14n111&Z&20220209~3122440&Z&...,NaN,L60D
1,P063/HP678806,63,NaN,NaN,0,M8008~M4856,330007&Z&20221005~330114&Z&20221011~311201&Z&2...,NaN,I69B
2,P017/HP223780,75,NaN,NaN,0,H058~E782~E790~H162~H405~H532~K219~M5496~N119~...,331102&B&20220103~331101&B&20220103,NaN,C63Z
3,P067/HP354818,50,NaN,NaN,0,D251~R103,8t901&Z&20220608~5q42714&Z&20220608,NaN,N21Z
4,P027/HP613843,0,1.0,2520.0,0,Z380~P050~Z138,NaN,NaN,P67D
...,...,...,...,...,...,...,...,...,...
49995,P019/HP471377,63,NaN,NaN,0,E871,NaN,NaN,K62B
49996,P054/HP551312,62,NaN,NaN,0,T435,8v311&Z&20220927,NaN,X62Z
49997,P031/HP332311,71,NaN,NaN,0,S3201~W019,3300023&Z&20220510~390000&Z&20220510~330008&Z&...,NaN,I68B
49998,P072/HP567440,28,NaN,NaN,0,O3430~D508~G408~O093~O990~O993~Z358,3131a&Z&20220118~5q230&Z&20220121,NaN,O05A


Po porovnaní dát s požadovaným formátom pre algoritmus a preštudovaní README pre verziu v2024.1 a aktuálnu verziu, som zistila, že dataset obsahuje dva stĺpce navyše, konkrétne stĺpce 2 a 7. Taktiež v stĺpci 3 (hmotnosť) by mala byť hodnota 0 namiesto NaN. V ďalších krokoch som tieto chyby opravila.

In [13]:
# Odstránenie stĺpcov s indexmi 2 a 7
new = data.drop(columns = [2,7])

In [23]:
# Vyplnenie chýbajúcich hodnôt (NaN) v stĺpci s indexom 3 hodnotou 0 a konverzia na celé čísla
new[3] = new[3].fillna(0).astype('int64')

In [25]:
# Uloženie upravených dát do nového CSV súboru bez indexov a hlavičky,
# s oddelením bodkočiarkami, aby mohli byť použité pre algoritmus
new.to_csv('new_data.csv', index=False, header=False, sep=';')

In [27]:
# Spustenie algoritmu z main vetvy so všetkými prepínačmi. 
%run ./main.py -v -n -d ./new_data.csv

Aktivovaný prepínač 'Všetky výkony hlavné'. Pri vyhodnotení príloh sa bude predpokladať, že ktorýkoľvek z výkazaných výkonov mohol byť hlavný.
Aktivovaný prepínač 'Vyhodnoť neúplné prípady'. V prípade, že nie je vyplnená nejaká povinná hodnota, aj tak sa bude pokračovať vo vyhodnocovaní.
Aktivovaný prepínač 'Ponechaj duplicity'. Vo výstupnom zozname medicínskych služieb budú ponechané aj duplicitné záznamy.


### Uloha 2 - Priradenie jednotlivým medicínskym službám ich úroveň. 

In [102]:
# Načítanie výsledkov z výstupného CSV súboru po spracovaní
outputs = pd.read_csv('new_data_output.csv', sep=';')

In [111]:
# Načítanie CSV súboru CSV súboru s definíciou úrovní medicínskych služieb z Prílohy č. 2 vyhlášky o kategorizácii ústavnej starostlivosti.
uroven_data = pd.read_csv('zoznam_ms_v2024.csv', sep=';')

In [104]:
# Rozdelenie hodnoty v stĺpci 'ms' na zoznamy, kde je použitý znak '~' ako oddeľovač
outputs['ms'] = outputs['ms'].str.split('~')

In [146]:
def assign_uroven(outputs, uroven):
    # Vytvorenie prázdneho stĺpca 'ms_uroven' na ukladanie úrovní medicínskych služieb ako zoznamy
    outputs['ms_uroven'] = [[] for _ in range(len(outputs))]
    
    # Iterácia cez každý riadok v tabuľke 'outputs'
    for idx, row in outputs.iterrows():
        ms_values = []  # Zoznam na ukladanie úrovní pre aktuálny riadok
        
        # Podľa veku pacienta vyberieme správny stĺpec pre úroveň medicínskej služby
        if row['vek'] < 1:
            age_col = 'uroven_ms_deti_0'
        elif row['vek'] >= 1 and row['vek'] <= 6:
            age_col = 'uroven_ms_deti_1'
        elif row['vek'] > 6 and row['vek'] <= 15:
            age_col = 'uroven_ms_deti_7'
        elif row['vek'] > 16 and row['vek'] <= 18:
            age_col = 'uroven_ms_deti_16'
        else: 
            age_col = 'uroven_ms_dospeli'
        
        # Iterácia cez zoznam medicínskych služieb (ms) pre konkrétny riadok
        for ms in row['ms']:
            # Vyhľadanie medicínskej služby v tabuľke 'uroven'
            ms_row = uroven[uroven['kod_ms'] == ms]
            
            # Ak nájdeme zodpovedajúci záznam, získame úroveň pre danú vekovú kategóriu
            value = ms_row[age_col].values[0] if not ms_row.empty else None
            ms_values.append(value)  # Pridanie úrovne do zoznamu
        
        # Uloženie zoznamu úrovní do stĺpca 'ms_uroven' pre aktuálny riadok
        outputs.at[idx, 'ms_uroven'] = ms_values
        
    return outputs

In [148]:
# Zavolanie funkcie na priradenie úrovní medicínskym službám
assign_uroven(outputs,uroven_data)

,id,vek,hmotnost,umela_plucna_ventilacia,diagnozy,vykony,drg,ms,ms_uroven
0,P019/HP663449,80,0,0,N178~D508~E1160~I1000~I480~N183,14n112&Z&20220209~14n111&Z&20220209~3122440&Z&...,L60D,"[S28-66, S28-66, S30-08]","[1.0, 1.0, 2.0]"
1,P063/HP678806,63,0,0,M8008~M4856,330007&Z&20221005~330114&Z&20221011~311201&Z&2...,I69B,[S29-13],[2.0]
2,P017/HP223780,75,0,0,H058~E782~E790~H162~H405~H532~K219~M5496~N119~...,331102&B&20220103~331101&B&20220103,C63Z,[S04-29],[3.0]
3,P067/HP354818,50,0,0,D251~R103,8t901&Z&20220608~5q42714&Z&20220608,N21Z,"[S10-37, S10-23]","[1.0, 2.0]"
4,P027/HP613843,0,2520,0,Z380~P050~Z138,NaN,P67D,[S48-04],[2.0]
...,...,...,...,...,...,...,...,...,...
49995,P019/HP471377,63,0,0,E871,NaN,K62B,[S25-14],[2.0]
49996,P054/HP551312,62,0,0,T435,8v311&Z&20220927,X62Z,[S25-12],[2.0]
49997,P031/HP332311,71,0,0,S3201~W019,3300023&Z&20220510~390000&Z&20220510~330008&Z&...,I68B,[S02-59],[2.0]
49998,P072/HP567440,28,0,0,O3430~D508~G408~O093~O990~O993~Z358,3131a&Z&20220118~5q230&Z&20220121,O05A,"[S11-20, S11-07]","[2.0, 2.0]"


Na začiatku som načítala dva CSV súbory: jeden obsahujúci výstupné dáta po spracovaní a druhý s definíciou úrovní medicínskych služieb z Prílohy č. 2 vyhlášky o kategorizácii ústavnej starostlivosti. Kódy medicínskych služieb v stĺpci ms boli v niektorých prípadoch spojené znakom ~, čo znamenalo, že sa vyskytovalo viac služieb v jednom riadku. Preto som tento stĺpec najprv rozdelila na zoznamy, aby sa s nimi dalo ľahšie pracovať.

Následne som vytvorila funkciu assign_uroven, ktorá priraďuje úroveň každej medicínskej službe na základe veku pacienta. Funkcia najprv pridá prázdny stĺpec ms_uroven, kde budú uložené úrovne jednotlivých služieb. Pre každý riadok v dátach sa najprv na základe veku pacienta určí, ktorý stĺpec s úrovňami má byť použitý. Pre deti mladšie ako 1 rok sa použil stĺpec uroven_ms_deti_0, pre deti medzi 1 a 6 rokmi uroven_ms_deti_1, pre deti medzi 7 a 15 rokmi uroven_ms_deti_7, pre dospievajúcich od 16 do 18 rokov uroven_ms_deti_16, a pre dospelých nad 18 rokov uroven_ms_dospeli.

Potom som pre každú medicínsku službu v zozname (v jednom riadku) vyhľadala zodpovedajúcu službu v CSV tabuľke s úrovňami a priradila som správnu hodnotu podľa veku pacienta. Ak bola medicínska služba nájdená, jej úroveň som pridala do zoznamu, a následne sa tento zoznam uložil do nového stĺpca ms_uroven pre daný riadok.

### Uloha 3 - Nájdi všetky prípady, kde úroveň hlavnej medicínskej služby je nižšia, než úroveň niektorej z ostatných medicínskych služieb. 

In [203]:
def find_wrong_order(outputs):
    wrong_order_indicies = []  # Zoznam na uloženie indexov riadkov s nesprávnym poradím úrovní
    wrong_order_count = 0  # Počítadlo prípadov s nesprávnym poradím

    # Iterácia cez každý riadok v tabuľke 'outputs'
    for idx, row in outputs.iterrows():
        # Pokračujeme iba ak má riadok aspoň dve medicínske služby s pridelenými úrovňami
        if len(row['ms_uroven']) < 2:
            continue 
        
        # Uložíme si prvú úroveň medicínskej služby
        first_val = row['ms_uroven'][0]
        
        # Ak je prvá úroveň None (nie je definovaná), pokračujeme na ďalší riadok
        if first_val is None:
            continue 
        
        # Prechádzame zvyšné úrovne medicínskych služieb v riadku
        for val in row['ms_uroven'][1:]:
            # Ak je prvá úroveň nižšia než ktorákoľvek z nasledujúcich, zaznamenáme index riadku
            if first_val < val:
                wrong_order_indicies.append(idx)  # Pridanie indexu riadku do zoznamu
                wrong_order_count += 1  # Zvýšenie počítadla nesprávnych prípadov
                break  # Ukončíme kontrolu tohto riadku, ak sme našli nesprávne poradie
        
    return wrong_order_count, wrong_order_indicies  # Vrátime počet prípadov a zoznam indexov

In [208]:
# Volanie funkcie na nájdenie prípadov s nesprávnym poradím
count, indices = find_wrong_order(outputs)

print(f"Počet všetkých prípadov, kde úroveň hlavnej medicínskej služby je nižšia, než úroveň niektorej z ostatných medicínskych služieb: {count}\n"
      f"List indexov všetkých prípadov: {indices}")

Počet všetkých prípadov, kde úroveň hlavnej medicínskej služby je nižšia, než úroveň niektorej z ostatných medicínskych služieb: 7620
List indexov všetkých prípadov: [0, 3, 11, 18, 20, 23, 29, 31, 36, 51, 55, 64, 70, 83, 89, 97, 107, 115, 118, 122, 124, 130, 137, 155, 158, 159, 168, 170, 179, 183, 189, 192, 196, 200, 209, 220, 222, 228, 230, 255, 257, 275, 288, 292, 293, 298, 300, 302, 315, 324, 325, 330, 334, 341, 347, 351, 357, 365, 373, 388, 389, 390, 391, 393, 411, 420, 430, 432, 444, 447, 463, 464, 468, 479, 487, 490, 495, 497, 502, 524, 530, 531, 546, 555, 556, 564, 570, 573, 574, 576, 585, 588, 601, 615, 621, 641, 642, 646, 653, 660, 664, 666, 673, 678, 689, 690, 709, 719, 721, 722, 723, 727, 729, 738, 748, 757, 764, 767, 800, 806, 809, 812, 835, 841, 846, 847, 848, 854, 864, 867, 876, 879, 895, 896, 901, 914, 923, 929, 934, 937, 941, 943, 948, 949, 952, 960, 963, 981, 987, 994, 1005, 1006, 1010, 1011, 1016, 1018, 1019, 1028, 1031, 1034, 1037, 1047, 1059, 1060, 1066, 1067, 1071,

Na vyriešenie tejto úlohy som vytvorila funkciu find_wrong_order, ktorá prechádza každý riadok dát uložených v tabuľke outputs. Najprv funkcia skontroluje, či má riadok aspoň dve medicínske služby. Ak má menej než dve služby, riadok sa preskočí, keďže nie je možné porovnávať úrovne.

Následne som z každého platného riadku získala prvú hodnotu zo stĺpca ms_uroven, ktorá reprezentuje hlavnú medicínsku službu. Ak je hodnota tejto služby prázdna alebo nedefinovaná (None), riadok sa opäť preskočí. Potom som porovnala túto hlavnú úroveň s úrovňami ostatných medicínskych služieb v danom riadku. Ak bola úroveň hlavnej služby nižšia ako niektorá z ostatných, index tohto riadku som priradila do zoznamu wrong_order_indicies a zároveň som zvýšila počítadlo nesprávnych prípadov.

Výsledkom tejto funkcie je zoznam indexov riadkov, v ktorých bola hlavná úroveň nižšia než ostatné, a celkový počet takýchto prípadov. V tomto prípade bolo zistených 7620 prípadov s nesprávnym poradím medicínskych služieb.

### Uloha 4 - nedokončená

Úlohu 4 som nestihla dokončiť, ale začala som tým, že som odstránila prípady, kde je iba jedna medicínska služba.

In [244]:
filtered_outputs = outputs[outputs['ms'].apply(lambda x: len(x) >= 2)]